In [1]:
# dependencies
import pandas as pd
# dashboard
import plotly.express       as px
from   jupyter_dash         import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from   dash.dependencies    import Input, Output, State
# ad hoc functions
from   utils import *

In [2]:
##  data
# modeling
path             = 'https://raw.githubusercontent.com/yoselalberto/ia_proyecto_final/main/data/processed/celulares_procesados.csv'
# to show in interface
path_formateados = 'https://raw.githubusercontent.com/yoselalberto/ia_proyecto_final/main/data/processed/celulares_formato.csv'

In [3]:
# Load Data
df_inicio     = pd.read_csv(path)
df_formateado = pd.read_csv(path_formateados)

In [4]:
## parametros para modelar
# estas columnas serán ignoradas durante el modelado, pero serán mostradas en la recomendación
columnas_ignorar     = {'color', 'pantalla'}
# variable objetivo
columna_objetivo     = 'producto_nombre'
# columnas categoricas
columnas_categoricas = ['marca', 'procesador', 'sistema_operativo', 'tecnologia']
# columnas numericas
columnas_numericas   = ['peso', 'camara_trasera', 'camara_frontal', 'ram', 'memoria', 'precio']
# variables predictoras
columnas_predictoras = columnas_numericas + columnas_categoricas

In [5]:
# elimino columnas, duplicados, y reordeno las columnas
df = df_inicio.drop(columns = columnas_ignorar).drop_duplicates().reset_index(drop = True)[columnas_predictoras + [columna_objetivo]]

In [6]:
# train and test split
predictores = df.drop(columns = columna_objetivo)
objetivo    = df[[columna_objetivo]]

## Ingeniería de variables

Para llenar valores faltantes usaremos el promedio para las variables númericas, y la moda para las variables categoricas; también estandarizamos las variables númericas, restamos la media, y dividimos entre la desviación estandar.  
Para la variable objetivo, simplemente aplicamos el one hot encoding.

In [7]:
# transformo los predictores
predictores_transformed = feature_engineer(predictores, columnas_numericas, columnas_categoricas)
# tambien la variable objetivo, one hot encoding
(objetivo_transformed, encoder_objetivo) = ohe_objetivo(objetivo)

## Modelado

Al final implementaremos un DecisionTreeClassifier, sus recomendaciones son subjetivamente mejor (el random forest recomendaba un modelo el 95% de las veces).

In [8]:
# creo evaluador para el grid search, utilizaré el micro f1 score
scorer_f1  = make_scorer(f1_score, average = 'micro')
# valores a explorar
# parameters = {'max_depth': [2, 4, 8], 'criterion': ['gini', 'entropy'], 'min_samples_leaf': [1, 2, 4], 'n_estimators': [5, 10, 20]}
parameters = {'max_depth': [2, 4, 8], 'criterion': ['gini', 'entropy'], 'min_samples_leaf': [1, 2, 4]}
# ajuste y evaluacion
# modelo = GridSearchCV(RandomForestClassifier(random_state = 59, oob_score = True, max_features = 6, class_weight = "balanced"), parameters, n_jobs = 6, scoring = scorer_f1, cv = 4)
modelo = GridSearchCV(DecisionTreeClassifier(random_state = 59, max_features = 6, class_weight = "balanced"), parameters, n_jobs = 6, scoring = scorer_f1, cv = 5)
# ajuste modelo
modelo.fit(X = predictores_transformed, y = objetivo_transformed)
# extraigo el mejor
modelo_mejor = modelo.best_estimator_
# ojeada
print(modelo.best_score_, modelo.best_params_) 

0.38014705882352945 {'criterion': 'entropy', 'max_depth': 8, 'min_samples_leaf': 1}


## Interacción con el usuario

El usuario introducirá característias deseadas. 

In [9]:
# mostrar la salida en html
def generate_table(dataframe, max_rows = 2):
    #
    df_html = html.Table([
        html.Thead(html.Tr([html.Th(col) for col in dataframe.columns])),
        html.Tbody([html.Tr([html.Td(dataframe.iloc[i][col]) for col in dataframe.columns]) for i in range(min(len(dataframe), max_rows))])
    ])
    #
    return df_html

In [10]:
## interfaz
# style
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
# instance app
app = JupyterDash(__name__, external_stylesheets = external_stylesheets)
# layout
app.layout = html.Div([
    html.H1(children = 'Recomendación de un telefono inteligente',
           style={'vertical-align': 'top', 'margin-left': '3vw'}),
    html.Div(children='''
    A partir de tus preferencias te recomendaremos un telefono:
    ''', style={'vertical-align': 'top', 'margin-left': '3vw'}),
    # first row
    html.Div(children=[
        # first column of first row
        html.Div(children=[
            html.Label('Marca'),
            dcc.Dropdown(id = 'marca', options=[
                {'label': 'Motorola', 'value': 'motorola'},
                {'label': 'Samsung',  'value': 'samsung'},
                {'label': 'Apple',    'value': 'apple'},
                {'label': 'Xiaomi',   'value': 'xiaomi'},
                {'label': 'Huawei',   'value': 'huawei'},
                {'label': 'Nokia',    'value': 'nokia'},
                {'label': 'TCL',      'value': 'tcl'}
            ], value = ''),
        ], style={'display': 'inline-block', 'vertical-align': 'top', 'margin-left': '3vw', 'margin-top': '3vw', "width": "7vw"}),
        # second column of first row
        html.Div(children=[
            html.Label('Sistema Operativo'),
            dcc.RadioItems(id = 'sistema_operativo',
                options=[
                    {'label': 'Android', 'value': 'android'},
                    {'label': 'IOS',     'value': 'ios'}
                ], value = ''),
            
        ], style={'display': 'inline-block', 'vertical-align': 'top', 'margin-left': '3vw', 'margin-top': '3vw',
                 "width": "10vw"}),
        # third column of first row
        html.Div(children=[
            html.Label('Red'),
            dcc.RadioItems(id = 'tecnologia', options=[
                {'label': '4G',    'value': '4g'},
                {'label': '4GLte', 'value': '4glte'},
                {'label': '5G',    'value': '5g'}], value = ''
            ),
        ], style={'display': 'inline-block', 'vertical-align': 'top', 'margin-left': '3vw', 'margin-right': '3vw', 
                  'margin-top': '3vw', "width": "7vw"}),
        html.Div(children=[
            # fourth column of first row
            html.Label('Procesador'),
            dcc.Dropdown(id = 'procesador', 
                options=[
                    {'label': 'Qualcomm', 'value': 'qualcomm'},
                    {'label': 'Samsung',  'value': 'samsung'},
                    {'label': 'Apple',    'value': 'apple'},
                    {'label': 'Mediatek', 'value': 'mediatek'},
                    {'label': 'ARM',      'value': 'arm'}
                ], value = ''),
        ], style={'display': 'inline-block', 'vertical-align': 'top', 'margin-rigth': '3vw', 'margin-top': '3vw', "width": "7vw"}),
    ], className='row'),
    
    #################################################################################################
       # second row
    html.Div(children=[
        # first column of first row
        html.Div(children=[
            html.Label('Precio'),
                dcc.Input(id = "precio_pesos", type = "number", debounce = True, placeholder = '$'),
        ], style={'display': 'inline-block', 'vertical-align': 'top', 'margin-left': '3vw', 'margin-top': '3vw'}),
        # five
        html.Div(children=[
               html.Label('Ram'),dcc.Input(id = "ram", type = "number", debounce = True, placeholder = 'Gb'),
        ], style={'display': 'inline-block', 'vertical-align': 'top', 'margin-left': '3vw', 'margin-top': '3vw'}),
        # six
        html.Div(children=[
               html.Label('Memoria'), dcc.Input(id = "memoria", type = "number", debounce = True, placeholder = 'Gb'),
        ], style={'display': 'inline-block', 'vertical-align': 'top', 'margin-left': '3vw', 'margin-top': '3vw'}),
        # third column of first row
        html.Div(children=[
            html.Label('Camara trasera'), dcc.Input(id = "camara_trasera", type = "number", debounce = True, placeholder = 'Mp'),
        ], style={'display': 'inline-block', 'vertical-align': 'top', 'margin-left': '3vw','margin-top': '3vw'}),
        # fourth column of first row
        html.Div(children=[
            html.Label('Camara delantera'), dcc.Input(id = "camara_frontal", type = "number", debounce = True, placeholder = 'Mp'),
        ], style={'display': 'inline-block', 'vertical-align': 'top', 'margin-left': '3vw', 'margin-top': '3vw'}),
        # second column of first row
        html.Div(children=[
            html.Label('Peso'),dcc.Input(id = "peso", type = "number", debounce = True, placeholder = 'gramos'),
        ], style={'display': 'inline-block', 'vertical-align': 'top', 'margin-left': '3vw', 'margin-top': '3vw'}),
    ], className='row'),
        
    #################################################################################################
       # third row
    html.Div(children=[
        # first column of first row
        html.Div(children=[
            html.Button(id = 'button_submit', n_clicks = 0, children = 'Calcular')
        ], style={'display': 'inline-block', 'vertical-align': 'top', 'margin-left': '3vw', 'margin-top': '3vw'}),
    ], className='row'),
    # salida test
    html.Div(children = '''Nuestra recomendación:''', 
             style={'display': 'inline-block', 'vertical-align': 'top', 'margin-left': '3vw', 'margin-top': '3vw'}),
    html.Div(id = 'recomendation',
             style={'display': 'inline-block', 'vertical-align': 'top', 'margin-left': '3vw', 'margin-top': '3vw'}),
])
# interacción
@app.callback(
    Output('recomendation',    'children'),
    Input('button_submit',     'n_clicks'),
    State('precio_pesos',      'value'),
    State('ram',               'value'),
    State('memoria',           'value'),
    State('camara_trasera',    'value'),
    State('camara_frontal',    'value'),
    State('peso',              'value'),   
    State('marca',             'value'),
    State('procesador',        'value'),
    State('sistema_operativo', 'value'),
    State('tecnologia',        'value')
    )
def make_recommendation(button_submit, precio_pesos, ram, memoria, camara_trasera, camara_frontal, peso,
                        marca, procesador, sistema_operativo, tecnologia):
    # valor inicial
    if(button_submit == 0):
        salida = ''
    else:
        # los valores son posicionales
        df_raw = pd.DataFrame({'precio': precio_pesos, 'ram': ram, 'memoria': memoria, 'camara_trasera': camara_trasera,
                     'camara_frontal': camara_frontal, 'peso': peso, 'marca': marca, 'procesador': procesador,
                     'sistema_operativo': sistema_operativo, 'tecnologia': tecnologia}, index = [0])
        # substituyo Nones por nan
        df_imputed = df_raw.fillna(value = np.nan)
        df_imputed['peso'] = df_imputed['peso'] / 1000
        # predict, predict_phone
        df_prediccion = predict_phone(df_imputed, predictores, columnas_numericas, columnas_categoricas, 
                                      modelo_mejor, encoder_objetivo, df_formateado)
        # nombres
        df_prediccion.columns = ['Nombre', 'Marca', 'Color', 'Sistema operativo', 'Memoria', 'Ram', 'Precio', 'Camara Trasera',
                                 'Camara Frontal', 'Pantalla', 'Tecnologia', 'Procesador', 'Peso']
        # formato
        salida = generate_table(df_prediccion)
    #
    return salida
#
app.run_server(mode = 'external')

Dash app running on http://127.0.0.1:8050/
